In [ ]:
import os
import random
from ase.build import bulk, molecule, add_adsorbate, surface
from ase import Atoms
from ase.io.vasp import write_vasp
from ase.optimize import LBFGS
from ase.io import Trajectory
from ase.constraints import FixAtoms
from fairchem.core import OCPCalculator
import pandas as pd


# 参数配置
elements = ['Ag', 'Au', 'Cu', 'Pd', 'Pt']  # 示例元素（可根据需求修改）
slab_size = (6, 6, 3)
num_samples = 2000

# 吸附物配置（保持不变）
adsorbate_names = ['CH4', 'CH3', 'CO', 'CO2', 'CH', 'C', 'HCO', 'CH3O', 'H', 'H2', 'H2O', 'OH']
adsorbates = {name: molecule(name) for name in adsorbate_names}
adsorbates.update({
    'CH2': Atoms('CH2', positions=[[0,0,0], [0,0,1.09], [0,0,-1.09]]),
    'O': Atoms('O', positions=[[0,0,0]]),
    'HCOO': Atoms('HCOO', positions=[[0,0,0], [0,0,1.23], [0,0,-1.23], [0,0,1.0]]),
    'COOH': Atoms('COOH', positions=[[0,0,0], [0,0,1.23], [0,0,-1.23], [0,0.97,-1.23]]),
    'CH2O': Atoms('CH2O', positions=[[0,0,0], [0,0,1.21], [0,0.94,-0.54], [0,-0.94,-0.54]])
})

positions = [(6,6), (6,7), (5,7), (7,5), (5,6), (6,5), (5,5), (7,7)]

results = []

# 主循环
for i, (slab, composition) in enumerate(generate_hea_slabs(elements, slab_size, num_samples)):
    # 获取唯一的元素组合（按字母排序）
    unique_elements = sorted(set(composition))  # 去重后排序，例如 ['Hf', 'Nb', 'Ta', 'Ti', 'Zr']
    combo_name = "".join(unique_elements) + "-FCC"  # 生成组合名称如 HfNbTaTiZr-BCC

    # 创建组合专属文件夹
    combo_dir = os.path.join(main_output_dir, combo_name)
    os.makedirs(combo_dir, exist_ok=True)

    slab.set_pbc([True, True, False])

    for ads_name, ads in adsorbates.items():
        for pos in positions:
            # 文件路径
            traj_path = os.path.join(combo_dir, f"slab_{i+1}_{ads_name}_{pos[0]}_{pos[1]}.traj")
            poscar_filename = os.path.join(combo_dir, f"slab_{i+1}_{ads_name}_{pos[0]}_{pos[1]}.vasp")

            # 结构处理
            slab_ads = slab.copy()
            add_adsorbate(slab_ads, ads, height=1.8, position=pos)
            slab_ads.center(axis=2, vacuum=10)
            constraint = FixAtoms(indices=range(len(slab)))
            slab_ads.set_constraint(constraint)
            slab_ads.calc = calc

            # 优化并保存
            dyn = LBFGS(slab_ads, trajectory=traj_path)
            dyn.run(fmax=0.05, steps=200)
            energy = slab_ads.get_potential_energy()
            write_vasp(poscar_filename, slab_ads, direct=True, sort=True, vasp5=True)

            # 记录结果
            results.append({
                "slab_index": i+1,
                "element_combo": "-".join(unique_elements),  # 记录元素组合
                "adsorbate": ads_name,
                "position": pos,
                "energy": energy,
                "traj_file": traj_path,
                "poscar_file": poscar_filename
            })

    # 定期保存进度
    if (i + 1) % 50 == 0:
        df_results = pd.DataFrame(results)
        df_results.to_csv(os.path.join(main_output_dir, "energy_results.csv"), index=False)
        print(f"Processed {i+1} slabs. Current combo: {combo_name}")

# 最终保存
df_results = pd.DataFrame(results)
df_results.to_csv(os.path.join(main_output_dir, "energy_results.csv"), index=False)
print("All tasks completed.")

INFO:root:Checking local cache: pretrained_models for model EquiformerV2-83M-S2EF-OC20-2M
